In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

F

<module 'pyspark.sql.functions' from '/opt/conda/lib/python3.8/site-packages/pyspark/sql/functions.py'>

In [3]:
try:
    spark.stop()
except:
    pass

try:
    sc.stop()
except:
    pass

In [5]:
## так как я взял инфру из первого дз, спарк там запущен в докер контейнере и к нему присосаны два слейва - его 2 ноды воркеров
spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .config("spark.sql.adaptive.enabled", False) \
    .config("spark.sql.autoBroadcastJoinThreshold", -1) \
    .config("spark.sql.sources.bucketing.enabled", True) \
    .config("spark.executor.memory", "1g") \
    .config("spark.driver.memory", "2g") \
    .appName("hw1") \
    .getOrCreate()
    # .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.3") \

- Параметры по памяти определить самостоятельно
- Включение AQE или Auto Broadcast => не зачет

# Задание 1

В исходном файле представлена информация о сотрудниках, их подразделении и заработной плате.
1. Использовать Spark-сессию, которую мы использовали на семинарах или в ДЗ №1
2. Посчитать среднюю зарплату в каждом подразделении с применением Spark RDD
3. При решении задачи необходимо использовать функцию `aggregateByKey`

In [4]:
spark

sc = spark.sparkContext

### Решение

In [13]:
# читаем employees.csv из смаунченной /data
rdd = sc.textFile('file:///data/task1/employees.csv')
rdd.take(5)

['1,Козлов Алексей,Design,121846',
 '2,Иванов Дмитрий,Operations,89891',
 '3,Васильев Роман,Sales,120380',
 '4,Александров Андрей,Security,178169',
 '5,Лебедев Дмитрий,Admin,61106']

In [24]:

# 3. парсим строку в (department, salary)
def parse_line(line: str):
    parts = line.split(",")
    # ожидаем ровно 4 поля: id, ФИО, департамент, зарплата
    dept = parts[2].strip()
    salary = int(parts[3].strip())   # ВАЖНО: квадратные скобки!
    return (dept, salary)

dept_salary_rdd = rdd.map(parse_line)

dept_salary_rdd.take(5)

[('Design', 121846),
 ('Operations', 89891),
 ('Sales', 120380),
 ('Security', 178169),
 ('Admin', 61106)]

In [29]:
zero_value = (0, 0)


## reduce внутри партиции, acc - сюда считаем N зарплат внутри партиции и делаем count + 1, salary - новое значение 
def seq_op(acc, salary):
    s, c = acc
    return (s + salary, c + 1)


## с разных партиций собираем данные
def comb_op(acc1, acc2):
    s1, c1 = acc1
    s2, c2 = acc2
    return (s1 + s2, c1 + c2)

sum_count_by_dept = dept_salary_rdd.aggregateByKey(
    zero_value,
    seq_op,
    comb_op
)  


avg_salary_by_dept = sum_count_by_dept.mapValues(
    lambda sc_pair: sc_pair[0] / sc_pair[1]  # sum / count
)


avg_salary_by_dept.collect()

[('Design', 128973.73214285714),
 ('Sales', 120917.60869565218),
 ('Security', 123997.92537313433),
 ('Admin', 127531.125),
 ('R&D', 119201.8),
 ('Product', 128217.5076923077),
 ('Legal', 125467.89333333333),
 ('Finance', 128928.08474576271),
 ('QA', 117809.54838709677),
 ('Support', 120634.5375),
 ('IT', 124254.38888888889),
 ('Marketing', 130941.44155844155),
 ('Operations', 125065.85483870968),
 ('HR', 125414.59154929577),
 ('Logistics', 122105.28301886792)]

# Задание 2

## Входные данные

Таблица `transactions` - информация о длительности просмотра контента пользователями:
1. user_uid — уникальный идентификатор пользователя
2. element_uid — уникальный идентификатор контента
3. watched_time — время просмотра в секундах

Справочник `catalogue` - каталог с описанием контента и метаинформации по нему

P.S. Как вы можете заметить при просмотре данных по пользователями, нужный нам ключ для операции будет перекошен (90% строк представлены на фильм, очень популярный среди смотревших) - это нужно доказать, то есть описать проблему в датасетах с точки зрения обработки Spark

### Решение

In [8]:
transactions = spark.read \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv("file:///data/task2/skew_transactions.csv")

transactions.show(5)
transactions.printSchema()

+--------+-----------+------------+
|user_uid|element_uid|watched_time|
+--------+-----------+------------+
|  408794|       2714|        7583|
|  247546|       2714|        4450|
|   43556|       2714|        7569|
|  517299|       2714|           1|
|  143593|       2714|        7284|
+--------+-----------+------------+
only showing top 5 rows

root
 |-- user_uid: integer (nullable = true)
 |-- element_uid: integer (nullable = true)
 |-- watched_time: integer (nullable = true)



Показываем явный перекос, ключ 2714 встречается очень много раз
ну ещё можно догадаться какую оптимизацию нужно использовать из этой строки     .config("spark.sql.autoBroadcastJoinThreshold", -1) \

In [ ]:
transactions.groupBy("element_uid").count().orderBy(F.desc("count")).show(10)

+-----------+--------+
|element_uid|   count|
+-----------+--------+
|       2714|20496000|
|        747|   61272|
|       2639|   61119|
|       3783|   55228|
|       2245|   52942|
|       6127|   52168|
|      10061|   48993|
|       3916|   48243|
|       8771|   47480|
|       3336|   45646|
+-----------+--------+
only showing top 10 rows



In [16]:
import json
from pyspark.sql.types import (
    StructType, StructField,
    IntegerType, StringType, ArrayType, DoubleType
)

# 1. Читаем JSON в Python (он небольшой, 2.8 МБ)
with open("/data/task2/catalogue.json", "r") as f:
    raw = json.load(f)   # dict: {"1983": {...}, "3783": {...}, ...}

# 2. Готовим строки с ОДНОЗНАЧНЫМИ типами
rows = []
for element_id_str, payload in raw.items():
    # ключ — это element_uid
    element_id = int(element_id_str)

    # защитно достаём значения, приводим к нужным типам
    t = payload.get("type")

    availability = payload.get("availability")
    if availability is not None:
        availability = [str(x) for x in availability]

    duration = payload.get("duration")
    duration = int(duration) if duration is not None else None

    def as_float(name):
        val = payload.get(name)
        return float(val) if val is not None else None

    feature_1 = as_float("feature_1")
    feature_2 = as_float("feature_2")
    feature_3 = as_float("feature_3")
    feature_4 = as_float("feature_4")
    feature_5 = as_float("feature_5")

    attributes = payload.get("attributes")
    if attributes is not None:
        attributes = [int(x) for x in attributes]

    row = {
        "element_uid": element_id,
        "type": t,
        "availability": availability,
        "duration": duration,
        "feature_1": feature_1,
        "feature_2": feature_2,
        "feature_3": feature_3,
        "feature_4": feature_4,
        "feature_5": feature_5,
        "attributes": attributes,
    }
    rows.append(row)

# 3. Явная схема
schema = StructType([
    StructField("element_uid", IntegerType(), False),
    StructField("type", StringType(), True),
    StructField("availability", ArrayType(StringType()), True),
    StructField("duration", IntegerType(), True),
    StructField("feature_1", DoubleType(), True),
    StructField("feature_2", DoubleType(), True),
    StructField("feature_3", DoubleType(), True),
    StructField("feature_4", DoubleType(), True),
    StructField("feature_5", DoubleType(), True),
    StructField("attributes", ArrayType(IntegerType()), True),
])

# 4. Создаём DataFrame с заданной схемой
catalogue = spark.createDataFrame(rows, schema=schema)

catalogue.printSchema()
catalogue.show(5, truncate=False)


root
 |-- element_uid: integer (nullable = false)
 |-- type: string (nullable = true)
 |-- availability: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- duration: integer (nullable = true)
 |-- feature_1: double (nullable = true)
 |-- feature_2: double (nullable = true)
 |-- feature_3: double (nullable = true)
 |-- feature_4: double (nullable = true)
 |-- feature_5: double (nullable = true)
 |-- attributes: array (nullable = true)
 |    |-- element: integer (containsNull = true)

+-----------+-----+------------------------------+--------+--------------------+------------+---------+------------+------------+-------------------------------------------------------------------------------------------+
|element_uid|type |availability                  |duration|feature_1           |feature_2   |feature_3|feature_4   |feature_5   |attributes                                                                                 |
+-----------+-----+-----------------------

In [18]:
# Здесь необходимо вывести результат:
# базовый join
joined_plain = (
    transactions.alias("t")
    .join(catalogue.alias("c"), on="element_uid", how="left")
)

joined_plain.show(50, truncate=False)

+-----------+--------+------------+------+------------------------------+--------+--------------------+------------+---------+------------+------------+---------------------------------------------------------------------------------------------------------------------------------------+
|element_uid|user_uid|watched_time|type  |availability                  |duration|feature_1           |feature_2   |feature_3|feature_4   |feature_5   |attributes                                                                                                                             |
+-----------+--------+------------+------+------------------------------+--------+--------------------+------------+---------+------------+------------+---------------------------------------------------------------------------------------------------------------------------------------+
|4101       |234775  |4506        |movie |[purchase, rent, subscription]|80      |3.7477159218975045E7|0.6773504396|16.0     |1.13860

### Решение с оптимизацией

In [ ]:
# Ваше решение в ячейках ниже ...

# оптимизированный join с broadcast
joined_optimized = (
    transactions.alias("t")
    .join(F.broadcast(catalogue).alias("c"), on="element_uid", how="left")
)

joined_optimized.explain()
joined_optimized.show(truncate=False)

== Physical Plan ==
*(2) Project [element_uid#92, user_uid#91, watched_time#93, type#329, availability#330, duration#331, feature_1#332, feature_2#333, feature_3#334, feature_4#335, feature_5#336, attributes#337]
+- *(2) BroadcastHashJoin [element_uid#92], [element_uid#328], LeftOuter, BuildRight, false
   :- FileScan csv [user_uid#91,element_uid#92,watched_time#93] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/data/task2/skew_transactions.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<user_uid:int,element_uid:int,watched_time:int>
   +- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, false] as bigint)),false), [plan_id=323]
      +- *(1) Scan ExistingRDD[element_uid#328,type#329,availability#330,duration#331,feature_1#332,feature_2#333,feature_3#334,feature_4#335,feature_5#336,attributes#337]


+-----------+--------+------------+-----+------------------------------+--------+--------------------+-------

# Задание 3

## Входные данные

Таблица `transactions`  — информация о длительности просомтра контента пользователями:

1. user_uid — уникальный идентификатор пользователя
2. element_uid — уникальный идентификатор контента
3. watched_time — время просмотра в секундах

Таблица `ratings`  — информация об оценках, поставленных пользователями:

1. user_uid — уникальный идентификатор пользователя
2. element_uid — уникальный идентификатор контента
3. rating — поставленный пользователем рейтинг

Справочник `user_uids`  — выборка пользователей:
1. user_uid — уникальный идентификатор пользователя


## Что нужно сделать
Для каждого пользователя из выборки посчитать:
1. Максимальное и минимальное время просмотра фильмов с оценками 8, 9 и 10
2. Название фичи должно быть в формате `feat_<агрегирующая_функция>_watched_time_rating_<оценка>`
3. Если у пользователь не ставил оценки 8, 9 и 10 то значение фичей должно быть null
4. Описать принятые при разработки кода решения и возможные оптимизации

Важно: сокращаем затраты на shuflle

## Перед выполнением всех join-ов, мы фильтруем оценки + делаем pivot для таблицы с оценкам

In [24]:
from pyspark.sql import functions as F

transactions = (
    spark.read
    .option("header", True)
    .csv("file:///data/task3/cut_transactions.csv")
    .select(
        F.col("user_uid").cast("long"),
        F.col("element_uid").cast("long"),
        F.col("watched_time").cast("long"),
    )
)

ratings = (
    spark.read
    .option("header", True)
    .csv("file:///data/task3/cut_ratings.csv")
    .select(
        F.col("user_uid").cast("long"),
        F.col("element_uid").cast("long"),
        F.col("rating").cast("int"),
    )
)

user_ids = (
    spark.read
    .option("header", True)
    .csv("file:///data/task3/ids.csv")
    .select(F.col("user_uid").cast("long"))
)

transactions.show(5)
ratings.show(5)
user_ids.show(5)


+--------+-----------+------------+
|user_uid|element_uid|watched_time|
+--------+-----------+------------+
|  563180|       6130|        3264|
|  133315|       2771|       10557|
|  290104|        984|        6845|
|  385385|       6980|        2734|
|  408794|       2714|        7583|
+--------+-----------+------------+
only showing top 5 rows

+--------+-----------+------+
|user_uid|element_uid|rating|
+--------+-----------+------+
|  571252|       1364|    10|
|   63140|       3037|    10|
|  443817|       4363|     8|
|  359870|       1364|    10|
|  359870|       3578|     9|
+--------+-----------+------+
only showing top 5 rows

+--------+
|user_uid|
+--------+
|  110138|
|  412991|
|   83691|
|  334052|
|   24124|
+--------+
only showing top 5 rows



### Решение

In [ ]:
# Ваше решение в ячейках ниже ...

In [ ]:
good_ratings = ratings.filter(F.col("rating").isin([8, 9, 10])) ## фильтрация оценок с 7 < x <= 10

In [27]:
joined = transactions.join(
    F.broadcast(good_ratings),
    on=["user_uid", "element_uid"],
    how="inner"
)

joined.show(5)

+--------+-----------+------------+------+
|user_uid|element_uid|watched_time|rating|
+--------+-----------+------------+------+
|   61690|       1415|        7289|     8|
|  566386|        747|       10979|     9|
|  153993|       1912|        6942|    10|
|  336070|       7987|        5434|     8|
|  485006|       7789|        7097|     9|
+--------+-----------+------------+------+
only showing top 5 rows



In [33]:
pivoted = (
    joined
        .groupBy("user_uid")
        .pivot("rating", [8, 9, 10])
        .agg(
            F.max("watched_time").alias("max_time"),
            F.min("watched_time").alias("min_time")
        )
)

pivoted.show(50)

+--------+----------+----------+----------+----------+-----------+-----------+
|user_uid|8_max_time|8_min_time|9_max_time|9_min_time|10_max_time|10_min_time|
+--------+----------+----------+----------+----------+-----------+-----------+
|  459856|      NULL|      NULL|      NULL|      NULL|       7799|       7799|
|   40132|      NULL|      NULL|      NULL|      NULL|       7566|       7566|
|  393270|      NULL|      NULL|     15287|     15287|       NULL|       NULL|
|  116139|      NULL|      NULL|      NULL|      NULL|      32916|      32916|
|  255354|      NULL|      NULL|      NULL|      NULL|        184|        184|
|  426340|      NULL|      NULL|      NULL|      NULL|      15819|      15819|
|  470763|      NULL|      NULL|      NULL|      NULL|       6868|       6868|
|  217853|      NULL|      NULL|      NULL|      NULL|       7682|       7682|
|  179448|      NULL|      NULL|      NULL|      NULL|       7902|       7902|
|  196029|      NULL|      NULL|      NULL|      NUL

In [34]:
final = user_ids.join(pivoted, on="user_uid", how="left")

final.show(truncate=False)

+--------+----------+----------+----------+----------+-----------+-----------+
|user_uid|8_max_time|8_min_time|9_max_time|9_min_time|10_max_time|10_min_time|
+--------+----------+----------+----------+----------+-----------+-----------+
|124821  |NULL      |NULL      |NULL      |NULL      |7271       |121        |
|537222  |15013     |15013     |NULL      |NULL      |NULL       |NULL       |
|107891  |5550      |93        |NULL      |NULL      |NULL       |NULL       |
|571326  |NULL      |NULL      |NULL      |NULL      |NULL       |NULL       |
|412991  |NULL      |NULL      |NULL      |NULL      |NULL       |NULL       |
|110138  |11682     |11682     |5160      |5160      |8599       |4569       |
|278352  |NULL      |NULL      |NULL      |NULL      |NULL       |NULL       |
|24124   |16        |16        |2578      |1575      |NULL       |NULL       |
|513130  |NULL      |NULL      |NULL      |NULL      |NULL       |NULL       |
|97634   |6504      |6504      |NULL      |NULL     

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 42684)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.8/socketserver.py", line 747, in __init__
    self.handle()
  File "/opt/conda/lib/python3.8/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/opt/conda/lib/python3.8/site-packages/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
  File "/opt/conda/lib/python3.8/site-packages/pyspark/accumulators.py", line 271, in accum_updates
    num_up

# Задание 4

## Входные данные

1. Потоковые данные о транзакциях из Kafka
2. Ивенты генерятся случайным образом, при помощи скрипта в директории task5/ или task4/ - они дублируются и структура одинаковая

Важно: данные генерируются случайным образом, поэтому результат будет у всех разный, соответственно можно ловить в том числе простые анамалии, например, статистически значимое отклонение значения какого-либо атрибута

## Что нужно сделать

1. Обработать потоковые данные: чтение и обработка данных о транзакциях в реальном времени, преобразовать входящие данные в DataFrame
2. Вычислить скользящее среднее значение суммы транзакций за последние 5 минут
3. Определить аномалии: пометить проблемные транзакции, например, если значение суммы транзакции больше среднего значения суммы транзакций в 2 раза
4. Вывести данные об аномальных транзакциях в консоль в режиме реального времени


In [ ]:
spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .config("spark.sql.adaptive.enabled", False) \
    .config("spark.sql.autoBroadcastJoinThreshold", -1) \
    .config("spark.sql.sources.bucketing.enabled", True) \
    .config("spark.executor.memory", "1g") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.3") \
    .config("spark.driver.memory", "2g") \
    .appName("hw1") \
    .getOrCreate()

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [8]:
spark

NameError: name 'spark' is not defined

In [26]:
spark.sparkContext.getConf().get("spark.jars.packages")

'org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.3'

In [28]:
spark.sparkContext.listJars()

AttributeError: 'SparkContext' object has no attribute 'listJars'

In [23]:
df = (
    spark.readStream
         .format("kafka")
         .option("kafka.bootstrap.servers", "kafka:29092")
         .option("subscribe", "transactions_stream")
         .load()
)

AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of Structured Streaming + Kafka Integration Guide.

In [14]:
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StructType, StructField,
    StringType, DoubleType, LongType
)

# 1. Описываем схему события
event_schema = StructType([
    StructField("transaction_id", StringType(), True),
    StructField("user_id", LongType(), True),
    StructField("timestamp", StringType(), True),  # сначала строкой
    StructField("amount", DoubleType(), True),
    StructField("currency", StringType(), True),
    StructField("transaction_type", StringType(), True),
    StructField("status", StringType(), True),
])

# 2. Читаем поток из Kafka
kafka_raw = (
    spark.readStream
         .format("kafka")
         .option("kafka.bootstrap.servers", "kafka:29092")
         .option("subscribe", "transactions_stream")  # имя топика из генератора
         .option("startingOffsets", "latest")
         .load()
)

# 3. Парсим value (JSON → столбцы)
parsed = kafka_raw.select(
    F.from_json(F.col("value").cast("string"), event_schema).alias("data")
).select("data.*")

# 4. Приводим timestamp к нормальному типу
events = parsed.withColumn(
    "event_time",
    F.to_timestamp("timestamp")
).drop("timestamp")

events.printSchema()


AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of Structured Streaming + Kafka Integration Guide.

# Задание 5

**Входные данные**

1. Вам будет предоставлен docker-compose файл. Отличный от инфраструкты для ДЗ№1 и предыдущих заданий этого ДЗ№2
2. Генератор ивентов для топика кафки. Из этого топика нужно будет считывать данные
3. Структура данных, которые будут поступать в топик
4. Скорипт с генерацие ивентов их записью в Kafka Topic

#### Описание входящих данных

transaction_id — уникальный идентификатор транзакции
user_id — идентификатор пользователя
timestamp — время транзакции
amount — сумма транзакции
currency — валюта транзакции
transaction_type — тип транзакции (покупка, возврат, снятие, пополнение)
status — статус транзакции (успешно, неуспешно, в ожидании)

**Что требуется сделать**

Представьте, что вам поступила задача. Необходимо по рилтайм данным собрать витрины для аналитиков, к которым они могли бы обращаться, поверх которых они могли бы строить дашборды и многое другое.
1. Ваша задача: перенести данные из контура Kafka в контур ClickHouse
2. Построить несколько Kafka Engine таблиц (здесь на ваше усмотрение, можно одну, а можно и несколько)
3. Учесть, что некоторые входящие данные содержать ошибки или могут не парситься и т д. Для решение этой проблемы необходимо реализовать deadletter механизм при помощи MV
4. У вас должно быть минимум две таблицы MergeTree, они должны отличаться и быть логичными, то есть содержать какой-то смысл, иными словами – быть удобными для пользователя
5. Необходимо приложить код кафка энжинов, код MV, код таблиц и скрины запросов и создания таблиц в ClickHouse

In [ ]:
# ваш код здесь ...

In [ ]:
# ваши скрипты создания Kafka-Engine здесь ...

In [ ]:
# ваши скрипты создания MV и MV DLQ здесь...

In [ ]:
# ваши скрипты создания MergeTree таблиц здесь ...

In [ ]:
# ваш прочий Python-код для решения задачи / аналитики / EDA здесь ...